In [1]:
import os
from dotenv import load_dotenv

# Load biến môi trường từ file .env (file .env cần nằm trong cùng thư mục với notebook)
load_dotenv()

# Lấy giá trị của TOGETHER_API_KEY từ biến môi trường
together_api_key = os.getenv('TOGETHER_API_KEY')


if not together_api_key:
    raise ValueError("Chưa tìm thấy TOGETHER_API_KEY trong file .env!")

elastic_api_key = os.getenv('ELASTIC_API_KEY')

if not elastic_api_key:
    raise ValueError("Chưa tìm thấy ELASTIC_API_KEY trong file .env!")

elastic_cloud_id = os.getenv('ELASTIC_CLOUD_ID')

if not elastic_cloud_id:
    raise ValueError("Chưa tìm thấy ELASTIC_CLOUD_ID trong file .env!")

print(" Key đã được thiết lập thành công!")


 Key đã được thiết lập thành công!


In [2]:
print(together_api_key)
print(elastic_api_key)
print(elastic_cloud_id)

ba4066853f6d39efbdb8e58274802acdb950955adb52d523b36cad88c5493aa1
LUVTcUlKWUIyR2JxbU84a1dRYVo6TFhidjQ4VlVRRmFYNUJQY2MweFVEUQ==
My_Elasticsearch_deployment:YXAtZWFzdC0xLmF3cy5lbGFzdGljLWNsb3VkLmNvbTo0NDMkNjAyNDBhMzU5NGU5NDIzNjkzMmJjYjQwMWYyZWZmMTMkNmQxNjEyNjAwZWI1NDQzZmE2ODA1YTI1YWQ5M2Y3ZTA=


In [3]:
from langchain_together import ChatTogether, TogetherEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
llm = ChatTogether(
    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo" 
)
#meta-llama/Llama-3.3-70B-Instruct-Turbo
result = llm.invoke("Thủ đô Việt Nam là thành phố nào? ")
print(result.content)

Thủ đô của Việt Nam là Hà Nội.


In [4]:
from tqdm.notebook import tqdm
embd = HuggingFaceEmbeddings(model_name="VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9140\2901307974.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embd = HuggingFaceEmbeddings(model_name="VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")
No sentence-transformers model found with name VoVanPhuc/sup-SimCSE-VietNamese-phobert-base. Creating a new one with mean pooling.


In [5]:
embd

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='VoVanPhuc/sup-SimCSE-VietNamese-phobert-base', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

## Load Elasticsearch Store



In [6]:
from langchain.vectorstores import ElasticsearchStore
index_name = "ecom-law0"

In [8]:
# Khởi tạo connection tới Elasticsearch thông qua Elastic Cloud
vector_store = ElasticsearchStore(
    es_cloud_id=elastic_cloud_id,
    es_api_key=elastic_api_key,
    index_name=index_name,
    embedding=embd,
)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9140\2227148762.py:2: LangChainPendingDeprecationWarning: The class `ElasticsearchStore` will be deprecated in a future version. Use :class:`~Use class in langchain-elasticsearch package` instead.
  vector_store = ElasticsearchStore(


In [9]:
indices = vector_store.client.cat.indices(format="json")
for index_info in indices:
    print(index_info["index"])


.internal.alerts-transform.health.alerts-default-000001
.internal.alerts-observability.logs.alerts-default-000001
.internal.alerts-observability.uptime.alerts-default-000001
.ds-metrics-fleet_server.agent_versions-default-2025.04.10-000001
.internal.alerts-ml.anomaly-detection.alerts-default-000001
.internal.alerts-observability.slo.alerts-default-000001
ecom-law0
.internal.alerts-default.alerts-default-000001
new-index
.internal.alerts-observability.apm.alerts-default-000001
.elastic-connectors-v1
.internal.alerts-observability.metrics.alerts-default-000001
.internal.alerts-ml.anomaly-detection-health.alerts-default-000001
.internal.alerts-observability.threshold.alerts-default-000001
.ds-metrics-fleet_server.agent_status-default-2025.04.10-000001
.elastic-connectors-sync-jobs-v1
.internal.alerts-security.alerts-default-000001
.ds-logs-enterprise_search.audit-default-2025.04.10-000001
.ds-logs-enterprise_search.api-default-2025.04.10-000001
.internal.alerts-stack.alerts-default-000001

In [10]:
num_docs = vector_store.client.count(index=index_name)['count']
print(f"Number of documents in the vectorstore: {num_docs}")


Number of documents in the vectorstore: 318


## Hybrid Retrieval

In [11]:
from typing import Dict
from langchain_elasticsearch import ElasticsearchRetriever
from elasticsearch import Elasticsearch

In [12]:
index_name = "ecom-law0"
text_field = "text"
es_url = "https://my-elasticsearch-deployment.es.ap-east-1.aws.elastic-cloud.com"

In [13]:
# --- Định nghĩa hàm hybrid_query ---
def hybrid_query(search_query: str) -> Dict:
    # Sử dụng embed_query để tạo vector cho truy vấn
    query_vector = embd.embed_query(search_query)
    return {
        "retriever": {
            "rrf": {  # Rank Reciprocal Fusion: kết hợp hai truy vấn theo kiểu "should"
                "retrievers": [
                    {  # Thành phần full text search: dùng match query trên trường "text"
                        "standard": {
                            "query": {
                                "match": {
                                    text_field: search_query
                                }
                            }
                        }
                    },
                    {  # Thành phần vector search: dùng knn query trên trường "embedding_field"
                        "knn": {
                            "field": "embedding_field",  # Điều chỉnh nếu mapping của bạn dùng tên khác
                            "query_vector": query_vector,
                            "k": 5,
                            "num_candidates": 10
                        }
                    }
                ]
            }
        }
    }


In [14]:
# --- Thiết lập Elasticsearch client với xác thực ---
from elasticsearch import Elasticsearch
headers = {"Authorization": "ApiKey " + elastic_api_key.strip()}
es_client = Elasticsearch(
    hosts=[es_url],
    headers=headers
)
print("ElasticSearch Cluster Info:")
print(es_client.info())

ElasticSearch Cluster Info:
{'name': 'instance-0000000001', 'cluster_name': '60240a3594e94236932bcb401f2eff13', 'cluster_uuid': 'nvcaKx7WRZCEdKsZ-3nr0w', 'version': {'number': '8.17.4', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'c63c7f5f8ce7d2e4805b7b3d842e7e792d84dda1', 'build_date': '2025-03-20T15:39:59.811110136Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [15]:
from langchain_elasticsearch import ElasticsearchRetriever
hybrid_retriever = ElasticsearchRetriever.from_es_params(
    index_name=index_name,
    body_func=hybrid_query,
    content_field=text_field,  # Trường chứa nội dung văn bản trong index
    url=es_url,
    api_key=elastic_api_key.strip()
)




## Prompting

In [52]:
from langchain.prompts import PromptTemplate
# --- Xây dựng prompt với retrieved context ---
prompt_template = """
Trước tiên, hãy phân tích câu hỏi sau đây và xác định các yếu tố pháp lý cốt lõi cần xem xét:
Câu hỏi: {query}

Sau đó, hãy đọc kỹ các thông tin pháp lý được trích xuất từ văn bản dưới đây:
------------------------------------------------
{retrieved_context}
------------------------------------------------

Dựa trên phân tích câu hỏi và các thông tin pháp lý trên, bạn là một chuyên gia (expert) có nhiều năm kinh nghiệm giảng dạy và tư vấn về Luật Thương Mại Điện Tử của Việt Nam. Nhiệm vụ của bạn là:
1. Phân tích cẩn thận câu hỏi dựa trên văn bản pháp lý, các điều khoản cụ thể và dẫn chứng hiện hành.
2. Xác định các yếu tố pháp lý quan trọng và trình bày các quy định liên quan, bao gồm trích dẫn chính xác từ bộ luật (ví dụ: "Điều X, khoản Y: ...") mà không được thay đổi nội dung gốc.
3. Nếu có nhiều quan điểm hoặc cách giải quyết khác nhau, hãy liệt kê và so sánh ưu, nhược điểm của từng phương án.
4. Đưa ra ví dụ minh họa cụ thể giúp làm rõ vấn đề.
5. Tổng hợp và tóm tắt các thông tin, sau đó trả lời câu hỏi một cách rõ ràng, logic và chi tiết.

Câu trả lời của bạn phải bao gồm các trích dẫn chính xác từ bộ luật và phải được trình bày theo cấu trúc rõ ràng, có tiêu đề phụ và giải thích chi tiết.

Trả lời:
"""


In [53]:
prompt = PromptTemplate(template=prompt_template, input_variables=["retrieved_context", "query"])

## Retrieval


### Test 1

In [18]:
# --- Thực hiện retrieval ---
query = "Theo luật trọng tài thương mại, trọng tài nước ngoài là gì?"
retrieved_docs = hybrid_retriever.invoke(query)

In [19]:
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])
print(retrieved_context)

18 CÔNG BÁO Số 1251 + 1252 Ngày 29-12-2015 Tòa án đã ra quyết định tuyên bố một người mất tích hoặc là đã chết có thẩm quyền giải quyết yêu cầu hủy bỏ quyết định tuyên bố một người mất tích hoặc là đã chết; d) Tòa án nơi người phải thi hành bản án, quyết định dân sự, hôn nhân và gia đình, kinh doanh, thương mại, lao động của Tòa án nước ngoài cư trú, làm việc, nếu người phải thi hành án là cá nhân hoặc nơi người phải thi hành án có trụ sở, nếu người phải thi hành án là cơ quan, tổ chức hoặc nơi có tài sản liên quan đến việc thi hành bản án, quyết định của Tòa án nước ngoài có thẩm quyền giải quyết yêu cầu công nhận và cho thi hành tại Việt Nam hoặc không công nhận bản án, quyết định dân sự, hôn nhân và gia đình, kinh doanh, thương mại, lao động của Tòa án nước ngoài; đ) Tòa án nơi người gửi đơn cư trú, làm việc, nếu người gửi đơn là cá nhân hoặc nơi người gửi đơn có trụ sở, nếu người gửi đơn là cơ quan, tổ chức có thẩm quyền giải quyết yêu cầu không công nhận bản án, quyết định dân sự,

In [20]:
final_prompt = prompt.format(retrieved_context=retrieved_context, query=query)

In [21]:
print(final_prompt)


Trước tiên, hãy phân tích câu hỏi sau đây và xác định các yếu tố pháp lý cốt lõi cần xem xét:
Câu hỏi: Theo luật trọng tài thương mại, trọng tài nước ngoài là gì?

Sau đó, hãy đọc kỹ các thông tin pháp lý được trích xuất từ văn bản dưới đây:
------------------------------------------------
18 CÔNG BÁO Số 1251 + 1252 Ngày 29-12-2015 Tòa án đã ra quyết định tuyên bố một người mất tích hoặc là đã chết có thẩm quyền giải quyết yêu cầu hủy bỏ quyết định tuyên bố một người mất tích hoặc là đã chết; d) Tòa án nơi người phải thi hành bản án, quyết định dân sự, hôn nhân và gia đình, kinh doanh, thương mại, lao động của Tòa án nước ngoài cư trú, làm việc, nếu người phải thi hành án là cá nhân hoặc nơi người phải thi hành án có trụ sở, nếu người phải thi hành án là cơ quan, tổ chức hoặc nơi có tài sản liên quan đến việc thi hành bản án, quyết định của Tòa án nước ngoài có thẩm quyền giải quyết yêu cầu công nhận và cho thi hành tại Việt Nam hoặc không công nhận bản án, quyết định dân sự, hôn nhân

In [22]:
response = llm(final_prompt)
print(response.content)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9140\3337770362.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(final_prompt)


**Trọng tài nước ngoài theo Luật Trọng tài thương mại**

Câu hỏi: Theo luật trọng tài thương mại, trọng tài nước ngoài là gì?

**Giải thích**

Theo Điều 3, khoản 11 của Luật Trọng tài thương mại, "Trọng tài nước ngoài là Trọng tài được thành lập theo quy định của pháp luật trọng tài nước ngoài do các bên thỏa thuận lựa chọn để tiến hành giải quyết tranh chấp ở ngoài lãnh thổ Việt Nam hoặc trong lãnh thổ Việt Nam."

**Phân tích**

Trọng tài nước ngoài là một hình thức trọng tài được thành lập theo quy định của pháp luật trọng tài nước ngoài, được các bên thỏa thuận lựa chọn để giải quyết tranh chấp. Điều này có nghĩa là trọng tài nước ngoài có thể được thành lập ở ngoài lãnh thổ Việt Nam hoặc trong lãnh thổ Việt Nam, nhưng phải tuân thủ quy định của pháp luật trọng tài nước ngoài.

**Ví dụ minh họa**

Ví dụ, hai công ty A và B có trụ sở tại Việt Nam và Nhật Bản, và họ có một tranh chấp về hợp đồng thương mại. Họ thỏa thuận lựa chọn trọng tài nước ngoài tại Nhật Bản để giải quyết tranh c

### Test 2

In [28]:

# --- Thực hiện retrieval ---
query = "Theo luật trọng tài thương mại, Quyết định trọng tài?"
retrieved_docs = hybrid_retriever.invoke(query)
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [29]:
final_prompt = prompt.format(retrieved_context=retrieved_context, query=query)

In [30]:
print(retrieved_context)

22 CÔNG BÁO Số 289 + 290 Ngày 29-05-2013 4. Thực hiện các quy định tại Mục 2 Chương II Nghị định này nếu website có chức năng đặt hàng trực tuyến cho các phiếu mua hàng, phiếu sử dụng dịch vụ hoặc thẻ khách hàng thường xuyên. 5. Thực hiện các quy định tại Mục 2 Chương V Nghị định này nếu website có chức năng thanh toán trực tuyến. 6. Công bố đầy đủ các nội dung quy định tại Điều 40 Nghị định này trong từng thông tin khuyến mại đăng tải trên website của mình. 7. Xây dựng và công bố trên website quy chế hoạt động tuân thủ các quy định tại Điều 38 Nghị định này. 8. Xây dựng và công bố trên website cơ chế tiếp nhận, giải quyết các khiếu nại của khách hàng về chất lượng hàng hóa, dịch vụ được khuyến mại hoặc hàng hóa, dịch vụ dùng để khuyến mại. 9. Chịu trách nhiệm của bên thứ ba trong việc cung cấp thông tin về hàng hóa, dịch vụ cho người tiêu dùng theo quy định của Luật Bảo vệ quyền lợi người tiêu dùng. 10. Bồi thường cho khách hàng nếu phiếu mua hàng, phiếu sử dụng dịch vụ hoặc thẻ khách

In [32]:
response = llm(final_prompt)
print(response.content)

**Phân tích câu hỏi và xác định các yếu tố pháp lý cốt lõi**

Câu hỏi đặt ra là "Theo luật trọng tài thương mại, Quyết định trọng tài?" Để trả lời câu hỏi này, chúng ta cần phân tích các yếu tố pháp lý cốt lõi liên quan đến quyết định trọng tài trong luật trọng tài thương mại.

**Các yếu tố pháp lý cốt lõi**

Theo Luật Trọng tài thương mại (Điều 1), trọng tài thương mại là phương thức giải quyết tranh chấp do các bên thỏa thuận và được tiến hành theo quy định của Luật này. Điều 2 của Luật này quy định về thẩm quyền giải quyết các tranh chấp của Trọng tài, bao gồm tranh chấp giữa các bên phát sinh từ hoạt động thương mại.

**Quyết định trọng tài**

Theo Điều 3 của Luật Trọng tài thương mại, quyết định trọng tài là quyết định cuối cùng của Trọng tài viên về tranh chấp giữa các bên. Điều này có nghĩa là quyết định trọng tài là kết quả cuối cùng của quá trình giải quyết tranh chấp bằng trọng tài.

**Thẩm quyền của Tòa án đối với quyết định trọng tài**

Theo Điều 39 của Luật Trọng tài thươn

### Test 3

In [39]:
# --- Thực hiện retrieval ---
query = '''Theo luật trọng tài thương mại,Đối với tranh chấp không có yếu tố nước ngoài, Hội đồng trọng tài có được áp dụng 
pháp luật Việt Nam để giải quyết tranh chấp hay không ?'''
retrieved_docs = hybrid_retriever.invoke(query)
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [40]:
final_prompt = prompt.format(retrieved_context=retrieved_context, query=query)

In [41]:
response = llm(final_prompt)
print(response.content)

**Phân tích câu hỏi và xác định các yếu tố pháp lý cốt lõi**

Câu hỏi đặt ra là: "Đối với tranh chấp không có yếu tố nước ngoài, Hội đồng trọng tài có được áp dụng pháp luật Việt Nam để giải quyết tranh chấp hay không?"

Để trả lời câu hỏi này, chúng ta cần xem xét các quy định liên quan đến trọng tài thương mại và pháp luật áp dụng trong giải quyết tranh chấp.

**Quy định về pháp luật áp dụng trong giải quyết tranh chấp**

Theo Điều 14 của Luật Trọng tài thương mại, quy định về pháp luật áp dụng trong giải quyết tranh chấp như sau:

"1. Đối với tranh chấp không có yếu tố nước ngoài, Hội đồng trọng tài áp dụng pháp luật Việt Nam để giải quyết tranh chấp.
2. Đối với tranh chấp có yếu tố nước ngoài, Hội đồng trọng tài áp dụng pháp luật do các bên lựa chọn; nếu các bên không có thỏa thuận về luật áp dụng thì Hội đồng trọng tài quyết định áp dụng pháp luật mà Hội đồng trọng tài cho là phù hợp nhất.
3. Trường hợp pháp luật Việt Nam, pháp luật do các bên lựa chọn không có quy định cụ thể liê

### Test 4

In [42]:
# --- Thực hiện retrieval ---
query = '''Luật trọng tài thương mại do ai ban hành'''
retrieved_docs = hybrid_retriever.invoke(query)
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [43]:
final_prompt = prompt.format(retrieved_context=retrieved_context, query=query)
response = llm(final_prompt)
print(response.content)

**Phân tích câu hỏi và xác định các yếu tố pháp lý cốt lõi**

Câu hỏi đặt ra là "Luật trọng tài thương mại do ai ban hành". Để trả lời câu hỏi này, chúng ta cần tìm hiểu về Luật Trọng tài thương mại và các quy định liên quan.

**Luật Trọng tài thương mại**

Luật Trọng tài thương mại là một bộ luật quy định về trọng tài thương mại, bao gồm các quy định về thành lập, hoạt động, và thẩm quyền của trọng tài thương mại.

**Quy định về ban hành Luật Trọng tài thương mại**

Theo quy định của pháp luật Việt Nam, Luật Trọng tài thương mại được ban hành bởi Quốc hội.

**Trích dẫn từ bộ luật**

Không có trích dẫn cụ thể từ bộ luật trong văn bản pháp lý cung cấp. Tuy nhiên, dựa trên kiến thức chung về pháp luật Việt Nam, chúng ta có thể kết luận rằng Luật Trọng tài thương mại được ban hành bởi Quốc hội.

**Kết luận**

Dựa trên phân tích và quy định của pháp luật, chúng ta có thể kết luận rằng Luật Trọng tài thương mại được ban hành bởi Quốc hội.

Tuy nhiên, cần lưu ý rằng văn bản pháp lý cung cấp 

### Test 5

In [44]:
# --- Thực hiện retrieval ---
query = '''Phán quyết của trọng tài trong luật trọng tài thương mại có hiệu lực như thế nào'''
retrieved_docs = hybrid_retriever.invoke(query)
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [45]:
final_prompt = prompt.format(retrieved_context=retrieved_context, query=query)
response = llm(final_prompt)
print(response.content)

**Phán quyết của trọng tài trong luật trọng tài thương mại có hiệu lực như thế nào**

**Khái quát về trọng tài thương mại**

Trọng tài thương mại là phương thức giải quyết tranh chấp do các bên thỏa thuận và được tiến hành theo quy định của Luật Trọng tài thương mại (Điều 3, Luật Trọng tài thương mại).

**Thẩm quyền của trọng tài thương mại**

Trọng tài thương mại có thẩm quyền giải quyết các tranh chấp phát sinh từ hoạt động thương mại, bao gồm:

* Tranh chấp giữa các bên phát sinh từ hoạt động thương mại (Điều 2, khoản 1, Luật Trọng tài thương mại)
* Tranh chấp phát sinh giữa các bên trong đó ít nhất một bên có hoạt động thương mại (Điều 2, khoản 2, Luật Trọng tài thương mại)
* Tranh chấp khác giữa các bên mà pháp luật quy định được giải quyết bằng Trọng tài (Điều 2, khoản 3, Luật Trọng tài thương mại)

**Hiệu lực của phán quyết trọng tài**

Phán quyết của trọng tài thương mại có hiệu lực thi hành ngay sau khi được ban hành, trừ trường hợp có kháng cáo hoặc kháng nghị (Điều 61, Luật 

### Test 6

In [46]:
# --- Thực hiện retrieval ---
query = '''Sự khác biệt giữa trọng tài thương mại và toà án là gì'''
retrieved_docs = hybrid_retriever.invoke(query)
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [47]:
final_prompt = prompt.format(retrieved_context=retrieved_context, query=query)
response = llm(final_prompt)
print(response.content)

**Sự khác biệt giữa trọng tài thương mại và tòa án**

Trọng tài thương mại và tòa án là hai phương thức giải quyết tranh chấp khác nhau trong lĩnh vực thương mại. Dưới đây là một số điểm khác biệt chính giữa hai phương thức này:

**1. Thẩm quyền**

* Trọng tài thương mại: Theo Điều 2 của Luật Trọng tài thương mại, trọng tài thương mại có thẩm quyền giải quyết các tranh chấp phát sinh từ hoạt động thương mại, bao gồm tranh chấp giữa các bên phát sinh từ hoạt động thương mại, tranh chấp phát sinh giữa các bên trong đó ít nhất một bên có hoạt động thương mại, và tranh chấp khác giữa các bên mà pháp luật quy định được giải quyết bằng trọng tài.
* Tòa án: Theo Điều 30 của Bộ luật Tố tụng dân sự, tòa án có thẩm quyền giải quyết các tranh chấp về kinh doanh, thương mại, bao gồm tranh chấp phát sinh trong hoạt động kinh doanh, thương mại giữa cá nhân, tổ chức có đăng ký kinh doanh với nhau và đều có mục đích lợi nhuận.

**2. Thủ tục**

* Trọng tài thương mại: Thủ tục trọng tài thương mại được 

### Test 7

In [48]:
# --- Thực hiện retrieval ---
query = '''Trong luật trọng tài thương mại, Phán quyết trọng tài có giá trị pháp lí như thế nào?'''
retrieved_docs = hybrid_retriever.invoke(query)
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [49]:
final_prompt = prompt.format(retrieved_context=retrieved_context, query=query)
response = llm(final_prompt)
print(response.content)

**Phán quyết trọng tài có giá trị pháp lí như thế nào?**

Theo Luật Trọng tài thương mại 2010, phán quyết trọng tài có giá trị pháp lí như sau:

* **Giá trị pháp lí của phán quyết trọng tài**: Phán quyết trọng tài có giá trị pháp lí ràng buộc các bên tranh chấp (Điều 61, Luật Trọng tài thương mại 2010).
* **Thi hành phán quyết trọng tài**: Phán quyết trọng tài được thi hành theo quy định của pháp luật về thi hành án dân sự (Điều 62, Luật Trọng tài thương mại 2010).
* **Không được kháng cáo**: Phán quyết trọng tài không được kháng cáo, trừ trường hợp có vi phạm nghiêm trọng về thủ tục tố tụng trọng tài (Điều 63, Luật Trọng tài thương mại 2010).

Tuy nhiên, cần lưu ý rằng phán quyết trọng tài chỉ có giá trị pháp lí khi đáp ứng các điều kiện sau:

* **Thỏa thuận trọng tài**: Các bên tranh chấp phải có thỏa thuận trọng tài hợp pháp (Điều 5, Luật Trọng tài thương mại 2010).
* **Tố tụng trọng tài**: Tố tụng trọng tài phải được tiến hành theo quy định của Luật Trọng tài thương mại 2010 (Điều 

### Test 8


In [59]:
# # --- Thực hiện retrieval ---
# query = "Theo luật trọng tài thương mại, Quyết định trọng tài?"
# retrieved_docs = hybrid_retriever.invoke(query)
# retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [58]:
# final_prompt = prompt.format(retrieved_context=retrieved_context, query=query)
# response = llm(final_prompt)
# print(response.content)

In [57]:
# print(final_prompt)